# 2. Preprocessing and Feature Engineering (Version 1)

## Overview
---
This section will be feature engineering the datasets by mapping ordinal variables and one hot encoding categorical variables. Two versions of datasets for modelling has been produced with different degrees of feature engineering. Both training and test sets (for Kaggle Submission) are engineered simultaneously. <br/>
#### Version 0
Description: 
- Mapping ordinal variables 
- One Hot Encoding categorical variables <br/>

Exported data: 
- train_model_v0.csv 
- test_model_v0.csv <br/>

#### Version 1 (Current Notebook)
Description: 
- Mapping ordinal variables
- One Hot Encoding categorical variables
- Reducing multicollinearity
- Mapping `Neighborhood` as ordinal variable <br/>

Exported data: 
- train_model_v1.csv
- test_model_v1.csv <br/>

### This notebook will be dealing with Version 1 only. 
Please refer to *02_Preprocessing_and_Feature_Engineering_v0* for Version 0.

## Importing Libraries
---

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
# allows us to see all rows and columns 
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
# limit floats to 3 decimal points

import numpy as np

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

## Load the Data
---

In [2]:
df_train = pd.read_csv('../data/train_clean_v1.csv')
df_train.head()

,id,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,lot_config,...,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,misc_val,mo_sold,yr_sold,sale_type,saleprice_log
0,109,60,RL,68.878999,13517,Pave,IR1,Lvl,AllPub,CulDSac,...,TA,TA,Y,0,44,0,3,2010,WD,11.779129
1,544,60,RL,43.000000,11492,Pave,IR1,Lvl,AllPub,CulDSac,...,TA,TA,Y,0,74,0,4,2009,WD,12.301383
2,153,20,RL,68.000000,7922,Pave,Reg,Lvl,AllPub,Inside,...,TA,TA,Y,0,52,0,1,2010,WD,11.599103
3,318,60,RL,73.000000,9802,Pave,Reg,Lvl,AllPub,Inside,...,TA,TA,Y,100,0,0,4,2010,WD,12.066811
4,255,50,RL,82.000000,14235,Pave,IR1,Lvl,AllPub,Inside,...,TA,TA,N,0,59,0,3,2010,WD,11.838626


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Data columns (total 67 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2049 non-null   int64  
 1   ms_subclass     2049 non-null   int64  
 2   ms_zoning       2049 non-null   object 
 3   lot_frontage    2049 non-null   float64
 4   lot_area        2049 non-null   int64  
 5   street          2049 non-null   object 
 6   lot_shape       2049 non-null   object 
 7   land_contour    2049 non-null   object 
 8   utilities       2049 non-null   object 
 9   lot_config      2049 non-null   object 
 10  land_slope      2049 non-null   object 
 11  neighborhood    2049 non-null   object 
 12  condition_1     2049 non-null   object 
 13  condition_2     2049 non-null   object 
 14  bldg_type       2049 non-null   object 
 15  house_style     2049 non-null   object 
 16  overall_qual    2049 non-null   int64  
 17  overall_cond    2049 non-null   i

In [4]:
df_test = pd.read_csv('../data/test_clean_v1.csv')
df_test.head()

,id,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,lot_config,...,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,misc_val,mo_sold,yr_sold,sale_type
0,2658,190,RM,69.000000,9142,Pave,Reg,Lvl,AllPub,Inside,...,440,Po,Po,Y,0,60,0,4,2006,WD
1,2718,90,RL,69.545961,9662,Pave,IR1,Lvl,AllPub,Inside,...,580,TA,TA,Y,170,0,0,8,2006,WD
2,2414,60,RL,58.000000,17104,Pave,IR1,Lvl,AllPub,Inside,...,426,TA,TA,Y,100,24,0,9,2006,New
3,1989,30,RM,60.000000,8520,Pave,Reg,Lvl,AllPub,Inside,...,480,Fa,TA,N,0,0,0,7,2007,WD
4,625,20,RL,69.545961,9500,Pave,IR1,Lvl,AllPub,Inside,...,514,TA,TA,Y,0,76,0,7,2009,WD


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 66 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              878 non-null    int64  
 1   ms_subclass     878 non-null    int64  
 2   ms_zoning       878 non-null    object 
 3   lot_frontage    878 non-null    float64
 4   lot_area        878 non-null    int64  
 5   street          878 non-null    object 
 6   lot_shape       878 non-null    object 
 7   land_contour    878 non-null    object 
 8   utilities       878 non-null    object 
 9   lot_config      878 non-null    object 
 10  land_slope      878 non-null    object 
 11  neighborhood    878 non-null    object 
 12  condition_1     878 non-null    object 
 13  condition_2     878 non-null    object 
 14  bldg_type       878 non-null    object 
 15  house_style     878 non-null    object 
 16  overall_qual    878 non-null    int64  
 17  overall_cond    878 non-null    int

In [6]:
# defining function for checking df shapes
def df_shape(df1, df2):
    print(df1.shape)
    print(df2.shape)
    
df_shape(df_train, df_test)

(2049, 67)
(878, 66)


In [7]:
# defining function for locating NaN values

def df_null_values(df):
    print(df.isnull().sum().sum())
    print(df.isnull().values.any())

def col_null_values(df, col):
    print(df[col].isnull().sum())
    print(df[col].isnull().values.any())

## Feature Engineering
---

### Binarize `Central Air` variable
>`Central Air` : Central air conditioning

Since the data values are Y/N, we can convert them to boolean variables. 

In [8]:
df_train['central_air'].value_counts()

Y    1908
N     141
Name: central_air, dtype: int64

In [9]:
df_train['central_air'] = df_train['central_air'].map({'Y': 1, 'N': 0})
df_train['central_air'].value_counts()

1    1908
0     141
Name: central_air, dtype: int64

In [10]:
df_test['central_air'].value_counts()

Y    823
N     55
Name: central_air, dtype: int64

In [11]:
df_test['central_air'] = df_test['central_air'].map({'Y': 1, 'N': 0})
df_test['central_air'].value_counts()

1    823
0     55
Name: central_air, dtype: int64

### Mapping Ordinal Variables

In [12]:
df_train['neighborhood'].unique()

array(['Sawyer', 'SawyerW', 'NAmes', 'Timber', 'Edwards', 'OldTown',
       'BrDale', 'CollgCr', 'Somerst', 'Mitchel', 'StoneBr', 'NridgHt',
       'Gilbert', 'Crawfor', 'IDOTRR', 'NWAmes', 'Veenker', 'MeadowV',
       'SWISU', 'NoRidge', 'ClearCr', 'Blmngtn', 'BrkSide', 'NPkVill',
       'Blueste', 'GrnHill', 'Greens', 'Landmrk'], dtype=object)

In [13]:
df_test['neighborhood'].unique()

array(['OldTown', 'Sawyer', 'Gilbert', 'NAmes', 'MeadowV', 'CollgCr',
       'Somerst', 'Mitchel', 'SawyerW', 'IDOTRR', 'BrkSide', 'Edwards',
       'ClearCr', 'NWAmes', 'Timber', 'NoRidge', 'NridgHt', 'Crawfor',
       'StoneBr', 'Veenker', 'BrDale', 'SWISU', 'Blmngtn', 'NPkVill',
       'Greens', 'Blueste'], dtype=object)

In [14]:
set(df_train['neighborhood']) -set(df_test['neighborhood'])

{'GrnHill', 'Landmrk'}

In [15]:
def convert_neighborhood(df):
    df['neighborhood'] = df['neighborhood'].map({'StoneBr': 4, 'NridgHt': 4, 'NoRidge': 4, 'GrnHill': 4, 
                                                 'Veenker': 4, 'Timber': 4, 'Somerst': 4, 
                                                 'ClearCr': 3, 'Crawfor': 3, 'CollgCr': 3, 'Blmngtn': 3, 
                                                 'NWAmes': 3, 'Gilbert': 3, 'Greens': 3,
                                                 'SawyerW': 2, 'Mitchel': 2, 'NAmes': 2, 'Blueste': 2, 
                                                 'NPkVill': 2, 'Sawyer': 2, 'Landmrk': 2,
                                                 'SWISU': 1, 'Edwards': 1, 'BrkSide': 1, 'OldTown': 1, 
                                                 'BrDale': 1, 'IDOTRR': 1, 'MeadowV': 1,})
    return df

In [16]:
df_train = convert_neighborhood(df_train)

In [17]:
df_train['neighborhood'].unique()

array([2, 4, 1, 3], dtype=int64)

In [18]:
df_test = convert_neighborhood(df_test)

In [19]:
df_train['neighborhood'].unique()

array([2, 4, 1, 3], dtype=int64)

In [20]:
def convert_ordinal_variables(df):
    df['lot_shape'] = df['lot_shape'].map({'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3':1})
    df['utilities'] = df['utilities'].map({'AllPub': 4, 'NoSewr': 3, 'NoSeWa': 2, 'ELO':1})
    df['land_slope'] = df['land_slope'].map({'Gtl': 3, 'Mod': 2, 'Sev':1})
    df['exter_qual'] = df['exter_qual'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1})
    df['exter_cond'] = df['exter_cond'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1})
    df['bsmt_qual'] = df['bsmt_qual'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA':0, np.nan:0})
    df['bsmt_cond'] = df['bsmt_cond'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA':0, np.nan:0})
    df['bsmt_exposure'] = df['bsmt_exposure'].map({'Gd': 4, 'Av': 3, 'Mn': 2, 'No':1, 'NA':0, np.nan:0})
    df['bsmtfin_type_1'] = df['bsmtfin_type_1'].map({'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf':1, 'NA':0, np.nan:0})
    df['bsmtfin_type_2'] = df['bsmtfin_type_2'].map({'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf':1, 'NA':0, np.nan:0})
    df['heating_qc'] = df['heating_qc'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1})
    df['electrical'] = df['electrical'].map({'SBrkr': 5, 'FuseA': 4, 'FuseF': 3, 'FuseP': 2, 'Mix':1})
    df['kitchen_qual'] = df['kitchen_qual'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1})
    df['functional'] = df['functional'].map({'Typ': 8, 'Min1': 7, 'Min2': 6, 'Mod': 5, 'Maj1': 4, 'Maj2': 3, 'Sev': 2, 'Sal':1})
    df['garage_finish'] = df['garage_finish'].map({'Fin': 3, 'RFn': 2, 'Unf':1, 'NA':0, np.nan:0})
    df['garage_qual'] = df['garage_qual'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA':0, np.nan:0})
    df['garage_cond'] = df['garage_cond'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA':0, np.nan:0})
    df['paved_drive'] = df['paved_drive'].map({'Y': 3, 'P': 2, 'N':1})
    return df

In [21]:
df_train = convert_ordinal_variables(df_train)

In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Data columns (total 67 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2049 non-null   int64  
 1   ms_subclass     2049 non-null   int64  
 2   ms_zoning       2049 non-null   object 
 3   lot_frontage    2049 non-null   float64
 4   lot_area        2049 non-null   int64  
 5   street          2049 non-null   object 
 6   lot_shape       2049 non-null   int64  
 7   land_contour    2049 non-null   object 
 8   utilities       2049 non-null   int64  
 9   lot_config      2049 non-null   object 
 10  land_slope      2049 non-null   int64  
 11  neighborhood    2049 non-null   int64  
 12  condition_1     2049 non-null   object 
 13  condition_2     2049 non-null   object 
 14  bldg_type       2049 non-null   object 
 15  house_style     2049 non-null   object 
 16  overall_qual    2049 non-null   int64  
 17  overall_cond    2049 non-null   i

In [23]:
df_null_values(df_train)

113
True


In [24]:
df_test = convert_ordinal_variables(df_test)

In [25]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 66 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              878 non-null    int64  
 1   ms_subclass     878 non-null    int64  
 2   ms_zoning       878 non-null    object 
 3   lot_frontage    878 non-null    float64
 4   lot_area        878 non-null    int64  
 5   street          878 non-null    object 
 6   lot_shape       878 non-null    int64  
 7   land_contour    878 non-null    object 
 8   utilities       878 non-null    int64  
 9   lot_config      878 non-null    object 
 10  land_slope      878 non-null    int64  
 11  neighborhood    878 non-null    int64  
 12  condition_1     878 non-null    object 
 13  condition_2     878 non-null    object 
 14  bldg_type       878 non-null    object 
 15  house_style     878 non-null    object 
 16  overall_qual    878 non-null    int64  
 17  overall_cond    878 non-null    int

In [26]:
df_null_values(df_test)

44
True


### OHE Categorical Variables

In [27]:
def convert_nominal_variables(df):
    df_ms_subclass = pd.get_dummies(df['ms_subclass'], prefix = 'ms_subclass', drop_first = True)
    df_ms_zoning = pd.get_dummies(df['ms_zoning'], prefix = 'ms_zoning', drop_first = True)
    df_street = pd.get_dummies(df['street'], prefix = 'street', drop_first = True)
    df_land_contour = pd.get_dummies(df['land_contour'], prefix = 'land_contour', drop_first = True)
    df_lot_config = pd.get_dummies(df['lot_config'], prefix = 'lot_config', drop_first = True)
    df_condition_1 = pd.get_dummies(df['condition_1'], prefix = 'condition_1', drop_first = True)
    df_condition_2 = pd.get_dummies(df['condition_2'], prefix = 'condition_2', drop_first = True)
    df_bldg_type = pd.get_dummies(df['bldg_type'], prefix = 'bldg_type', drop_first = True)
    df_house_style = pd.get_dummies(df['house_style'], prefix = 'house_style', drop_first = True)
    df_roof_style = pd.get_dummies(df['roof_style'], prefix = 'roof_style', drop_first = True)
    df_roof_matl = pd.get_dummies(df['roof_matl'], prefix = 'roof_matl', drop_first = True)
    df_exterior_1st = pd.get_dummies(df['exterior_1st'], prefix = 'exterior_1st', drop_first = True)
    df_exterior_2nd = pd.get_dummies(df['exterior_2nd'], prefix = 'exterior_2nd', drop_first = True)
    df_mas_vnr_type = pd.get_dummies(df['mas_vnr_type'], prefix = 'mas_vnr_type', drop_first = True)
    df_foundation = pd.get_dummies(df['foundation'], prefix = 'foundation', drop_first = True)
    df_heating = pd.get_dummies(df['heating'], prefix = 'heating', drop_first = True)
    df_garage_type = pd.get_dummies(df['garage_type'], prefix = 'garage_type', drop_first = True)
    df_sale_type = pd.get_dummies(df['sale_type'], prefix = 'sale_type', drop_first = True)
    df = pd.concat([df, df_ms_subclass, df_street, 
                    df_land_contour, df_lot_config, df_condition_1, 
                    df_condition_2, df_bldg_type, 
                    df_house_style, df_roof_style, 
                    df_roof_matl, df_exterior_1st,
                    df_exterior_2nd, df_mas_vnr_type,
                    df_foundation, df_heating,
                    df_garage_type, df_sale_type], axis = 1, join = 'outer')
    df.drop(['ms_zoning', 'ms_subclass', 
             'street', 'land_contour', 
             'lot_config', 
             'condition_1', 'condition_2', 
             'bldg_type', 'house_style',
             'roof_style', 'roof_matl',
             'exterior_1st', 'exterior_2nd',
             'mas_vnr_type', 'foundation',
             'heating', 'garage_type', 
             'sale_type'], axis=1, inplace=True)
    return df

In [28]:
df_train = convert_nominal_variables(df_train)

In [29]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Columns: 160 entries, id to sale_type_WD 
dtypes: float64(10), int64(39), uint8(111)
memory usage: 1006.6 KB


In [30]:
df_null_values(df_train)

0
False


In [31]:
df_test = convert_nominal_variables(df_test)

In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Columns: 154 entries, id to sale_type_WD 
dtypes: float64(3), int64(45), uint8(106)
memory usage: 420.3 KB


In [33]:
df_null_values(df_test)

0
False


In [34]:
df_shape(df_train, df_test)

(2049, 160)
(878, 154)


In [35]:
# checking the difference (non-symmetrical)
set(df_train.columns) - set(df_test.columns)

{'condition_2_Feedr',
 'condition_2_PosN',
 'condition_2_RRAe',
 'condition_2_RRAn',
 'condition_2_RRNn',
 'exterior_1st_CBlock',
 'exterior_1st_ImStucc',
 'exterior_1st_Stone',
 'exterior_2nd_Stone',
 'heating_OthW',
 'heating_Wall',
 'ms_subclass_150',
 'roof_matl_Membran',
 'saleprice_log'}

In [36]:
# checking the difference (non-symmetrical)
set(df_test.columns) - set(df_train.columns)

{'exterior_1st_PreCast',
 'exterior_2nd_Other',
 'exterior_2nd_PreCast',
 'heating_GasA',
 'mas_vnr_type_CBlock',
 'roof_matl_Metal',
 'roof_matl_Roll',
 'sale_type_VWD'}

In [37]:
# dropping columns in training set as the two datasets need to contain the same features
df_train = df_train.drop(columns=['condition_2_Feedr',
 'condition_2_PosN',
 'condition_2_RRAe',
 'condition_2_RRAn',
 'condition_2_RRNn',
 'exterior_1st_CBlock',
 'exterior_1st_ImStucc',
 'exterior_1st_Stone',
 'exterior_2nd_Stone',
 'heating_OthW',
 'heating_Wall',
 'ms_subclass_150',
 'roof_matl_Membran',])
df_train

,id,lot_frontage,lot_area,lot_shape,utilities,land_slope,neighborhood,overall_qual,overall_cond,year_built,...,garage_type_CarPort,garage_type_Detchd,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
0,109,68.878999,13517,3,4,3,2,6,8,1976,...,0,0,0,0,0,0,0,0,0,1
1,544,43.000000,11492,3,4,3,2,7,5,1996,...,0,0,0,0,0,0,0,0,0,1
2,153,68.000000,7922,4,4,3,2,5,7,1953,...,0,1,0,0,0,0,0,0,0,1
3,318,73.000000,9802,4,4,3,4,5,5,2006,...,0,0,0,0,0,0,0,0,0,1
4,255,82.000000,14235,3,4,3,2,6,8,1900,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,1587,79.000000,11449,3,4,3,4,8,5,2007,...,0,0,0,0,0,0,0,0,0,1
2045,785,68.878999,12342,3,4,3,1,4,5,1940,...,0,1,0,0,0,0,0,0,0,1
2046,916,57.000000,7558,4,4,3,3,6,6,1928,...,0,1,0,0,0,0,0,0,0,1
2047,639,80.000000,10400,4,4,3,2,4,5,1956,...,0,0,0,0,0,0,0,0,0,1


In [38]:
# dropping columns in testing set as the two datasets need to contain the same features
df_test = df_test.drop(columns=['exterior_1st_PreCast',
                                'exterior_2nd_Other',
                                'exterior_2nd_PreCast',
                                'heating_GasA',
                                'mas_vnr_type_CBlock',
                                'roof_matl_Metal',
                                'roof_matl_Roll',
                                'sale_type_VWD'])
df_test

,id,lot_frontage,lot_area,lot_shape,utilities,land_slope,neighborhood,overall_qual,overall_cond,year_built,...,garage_type_CarPort,garage_type_Detchd,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
0,2658,69.000000,9142,4,4,3,1,6,8,1910,...,0,1,0,0,0,0,0,0,0,1
1,2718,69.545961,9662,3,4,3,2,5,4,1977,...,0,0,0,0,0,0,0,0,0,1
2,2414,58.000000,17104,3,4,3,3,7,5,2006,...,0,0,0,0,0,0,0,1,0,0
3,1989,60.000000,8520,4,4,3,1,5,6,1923,...,0,1,0,0,0,0,0,0,0,1
4,625,69.545961,9500,3,4,3,2,6,5,1963,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,80.000000,8000,4,4,3,3,6,6,1974,...,0,0,0,0,0,0,0,0,0,1
874,1234,90.000000,14670,4,4,3,2,6,7,1966,...,0,0,0,0,0,0,0,0,0,1
875,1373,55.000000,8250,4,4,3,2,5,5,1968,...,0,0,0,0,0,0,0,0,0,1
876,1672,60.000000,9000,4,4,3,2,4,6,1971,...,0,1,0,0,0,0,0,0,0,1


In [39]:
df_shape(df_train, df_test)

(2049, 147)
(878, 146)


## Exporting Datasets for Modelling
---

In [40]:
df_train.to_csv('../data/train_model_v1.csv', index=False)

In [41]:
df_test.to_csv('../data/test_model_v1.csv', index=False)